In [1]:
from konlpy.tag import Twitter
from gensim.models import Word2Vec
import csv
from konlpy.tag import Twitter
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Okt
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import tensorflow as tf
import matplotlib

import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

import csv
import logging
from konlpy import jvm
logger = logging.getLogger(__name__)
jvm.init_jvm()



C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages

In [2]:
data_set.head()

NameError: name 'data_set' is not defined

In [48]:
data_set = pd.read_excel("data.xlsx")
data_set = data_set.iloc[:,1:]
data_set.rename({0:'news'},axis=1, inplace= True)
data_label = {1:'culture',2:'global',3:'politic',4:'society',5:'economy'}




In [49]:
for i in range(len(data_set)) :
    data_set['news'][i] = re.sub('[^a-zA-Z0-9 ㄱ-ㅣ가-힣]', '', data_set['news'][i])
    


C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [50]:
data_set = data_set.sample(frac=1).reset_index(drop=True)

In [51]:
data_set.head()

,news,label
0,질 자신 없다 농담하던 바둑천재AI 가슴 아팠다,1
1,행인 목 물고 뺨 때리고머슬마니아 만취 난동 당시,4
2,이낙연 착오 있었다강남 아파트 팔 것,3
3,손석희 하차 후폭풍MBC 사장설에 손 조용히 지내겠다,4
4,김정은 답방 부산으로 11월 아세안 초청 검토,3


In [52]:
def get_noun(text):
    tokenizer = Twitter()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

def get_noun2(text):
    tokenizer = Hannanum()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

def get_noun3(text):
    tokenizer = Okt()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

def get_morphs3(text):
    tokenizer = Okt()
    morphs = tokenizer.morphs(text)
    return [n for n in morphs]

def get_noun4(text):
    tokenizer = Komoran()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]


In [53]:

CV = CountVectorizer(tokenizer = get_noun3)
CV2 = CountVectorizer(tokenizer = get_morphs3)


In [54]:
CV = CountVectorizer(tokenizer = get_noun3)

In [55]:
train_set, test_set = train_test_split(data_set, test_size = 0.3, random_state = 42)


In [56]:
dic = { 
        '기자':'가',
        '연합뉴스': '가',
        '뉴스':'가',
        '사진': '가',
        '무단': '가',
        '금지':'가',
        '영상': '가',
        '동아일보':'가',
        '제공':'가',
        '전문':'가',
        '종합':'가',
        '단독':'가',
        '속보':'가',
        '이슈':'가',
        '속':'가'}

In [57]:
data_set = data_set.replace(dic,'')

In [58]:
fit_transform = CV.fit_transform(train_set["news"])

In [61]:
#data_set

In [62]:
#CV.vocabulary_

In [63]:
label_set = data_set['label']

In [64]:
from konlpy.tag import Twitter
from konlpy.tag import Kkma
twitter = Twitter()
token = []
embeddingmodel = []
for i in range(len(data_set)):
    label = data_set['label']
    sentence = twitter.pos(data_set['news'][i], norm=True, stem=True)
    temp = []
    temp_embedding = []
    all_temp = []
    for k in range(len(sentence)):
        temp_embedding.append(sentence[k][0])
        temp.append(sentence[k][0] + '/' + sentence[k][1])
    all_temp.append(temp)
    embeddingmodel.append(temp_embedding)
    
    all_temp.append(label[i])
    token.append(all_temp)
    
embeddingmodel = []
for i in range(len(label_set)):
    temp_embeddingmodel = []
    for k in range(len(token[i][0])):
        temp_embeddingmodel.append(token[i][0][k])
    embeddingmodel.append(temp_embeddingmodel)
embedding  = Word2Vec(embeddingmodel, size = 300, window =5, min_count = 10,
                      iter = 5, sg=1)

C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [65]:
def One_hot(data):
       
    index_dict = {value:index for index,value in enumerate(set(data))}
    result = []

    for value in data:

        one_hot = np.zeros(len(index_dict))
        index = index_dict[value]
        one_hot[index] = 1
        result.append(one_hot)

    return result


def Convert2Vec(model_name, doc): # Convert corpus into vectors
    #train_X_ = W2V.Convert2Vec("Word2Vec_csv_article.embedding",train_X)
    word_vec = []
    model = model_name
    for sent in doc:
        sub = []
        for word in sent:
            if word in model.wv.vocab:
                sub.append(model.wv[word]) # Word Vector Input
            else:
                sub.append(np.random.uniform(-0.25,0.25,300)) # used for OOV words
        word_vec.append(sub)

    return word_vec





In [66]:
class Bi_LSTM():
    
    def __init__(self, lstm_units, num_class, keep_prob):
        
        self.lstm_units = lstm_units
        
        with tf.variable_scope('forward', reuse = tf.AUTO_REUSE):
            
            self.lstm_fw_cell = tf.nn.rnn_cell.LSTMCell(lstm_units, forget_bias=1.0, state_is_tuple=True)
            self.lstm_fw_cell = tf.contrib.rnn.DropoutWrapper(self.lstm_fw_cell, output_keep_prob = keep_prob)
            
        with tf.variable_scope('backward', reuse = tf.AUTO_REUSE):
            
            self.lstm_bw_cell = tf.nn.rnn_cell.LSTMCell(lstm_units, forget_bias=1.0, state_is_tuple=True)
            self.lstm_bw_cell = tf.contrib.rnn.DropoutWrapper(self.lstm_fw_cell, output_keep_prob = keep_prob)
        
        with tf.variable_scope('Weights', reuse = tf.AUTO_REUSE):
           
            self.W = tf.get_variable(name="W", shape=[2 * lstm_units, num_class],
                                dtype=tf.float32, initializer = tf.contrib.layers.xavier_initializer())
            self.b = tf.get_variable(name="b", shape=[num_class], dtype=tf.float32,
                                initializer=tf.zeros_initializer())
            
            
    def logits(self, X, W, b, seq_len):
        
        (output_fw, output_bw), states = tf.nn.bidirectional_dynamic_rnn(self.lstm_fw_cell, self.lstm_bw_cell,dtype=tf.float32,
                                                                            inputs = X, sequence_length = seq_len)
        ## concat fw, bw final states
        outputs = tf.concat([states[0][1], states[1][1]], axis=1)
        pred = tf.matmul(outputs, W) + b        
        return pred
        
    def model_build(self, logits, labels, learning_rate = 0.001):
        
        with tf.variable_scope("loss"):
            
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits , labels = labels)) # Softmax loss
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss) # Adam Optimizer
            
        return loss, optimizer
    
    def graph_build(self, avg_loss, avg_acc):
        
        tf.summary.scalar('Loss', avg_loss)
        tf.summary.scalar('Accuracy', avg_acc)
        merged = tf.summary.merge_all()
        return merged

In [75]:
def Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size):
        
    zero_pad = np.zeros((Batch_size, Maxseq_length, Vector_size))
    for i in range(Batch_size):
        zero_pad[i, :np.shape(train_batch_X[i])[0], :np.shape(train_batch_X[i])[1] ] = train_batch_X[i]
    return zero_pad

In [76]:
tokens = np.array(token)
train_x = tokens[:,0]
train_y = tokens[:,1]

train_y = One_hot(train_y)
train_x = Convert2Vec(embedding, train_x)

In [77]:
Batch_size = 32
Total_size = len(train_x)
Vector_size = 300
seq_length = [len(x) for x in train_x]
Maxseq_length = max(seq_length)
learning_rate = 0.001
lstm_units = 128
num_class = 5
training_epochs = 5
keep_prob = 0.75

X = tf.placeholder(tf.float32, shape = [None, Maxseq_length, Vector_size])
Y = tf.placeholder(tf.float32, shape = [None, num_class])
seq_len = tf.placeholder(tf.int32 , shape = [None])

BiLSTM = Bi_LSTM(lstm_units, num_class, keep_prob)
with tf.variable_scope('loss', reuse = tf.AUTO_REUSE) :
    logits = BiLSTM.logits(X, BiLSTM.W, BiLSTM.b, seq_len)
    loss, optimizer = BiLSTM.model_build(logits, Y, learning_rate)

prediction = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

total_batch = int(Total_size / Batch_size)

print("Start training!")

#modelName = "BiLSTM.model"
saver = tf.train.Saver()

Start training!


In [78]:
import time
with tf.Session() as sess:

    start_time = time.time()
    sess.run(init)
    train_writer = tf.summary.FileWriter('Bidirectional_LSTM', sess.graph)
    i = 0
    for epoch in range(training_epochs):

        avg_acc, avg_loss = 0. , 0.
        for step in range(total_batch):

            train_batch_X = train_x[step*Batch_size : step*Batch_size+Batch_size]
            
            train_batch_Y = train_y[step*Batch_size : step*Batch_size+Batch_size]
            batch_seq_length = seq_length[step*Batch_size : step*Batch_size+Batch_size]
            train_batch_X = Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size)
            
            sess.run(optimizer, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
            # Compute average loss
            loss_ = sess.run(loss, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
            avg_loss += loss_ / total_batch
            
            acc = sess.run(accuracy , feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
            avg_acc += acc / total_batch
        print("epoch : {:02d} step : {:04d} loss = {:.6f} accuracy= {:.6f}".format(epoch+1, step+1, loss_, acc))

        summary = sess.run(BiLSTM.graph_build(avg_loss, avg_acc))       
        train_writer.add_summary(summary, i)
        i += 1

    duration = time.time() - start_time
    minute = int(duration / 60)
    second = int(duration) % 60
    print("%dminutes %dseconds" % (minute,second))
    save_path = saver.save(sess, os.getcwd())

    train_writer.close()
    print('save_path',save_path)

IndexError: tuple index out of range

In [99]:
#Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size)
#print(train_batch_X[1].shape)
zero_pad = np.zeros((Batch_size, Maxseq_length, Vector_size))
print(zero_pad.shape)
print(np.shape(train_batch_X[0]))

#print(zero_pad.shape[0,list(np.shape(train_batch_X[0]))[0],:])
a,b = 0,0
for i in range(Batch_size):
    a= int(np.shape(train_batch_X[i])[0])
    b= int(np.shape(train_batch_X[i])[1])
    zero_pad[i,:a,:b] = train_batch_X[i]
zero_pad[:10]

(32, 19, 300)
(12, 300)


ValueError: cannot copy sequence with size 12 to array axis with dimension 0

In [98]:
np.shape(train_batch_X[1])[1]

300

In [ ]:
np.asarray(train_batch_X[0]).shape[1]

In [81]:
int(np.shape(train_batch_X[0])[1])

300

In [ ]:
print(type(list(zero_pad.shape)))

In [ ]:
print(type(np.shape(train_batch_X[0])[0]))

In [ ]:
print(type(list(np.shape(train_batch_X[0]))[0]))

In [ ]:
zero_pad[0,:12,:300]

In [ ]:
np.asarray(train_batch_X[1]).shape

In [ ]:
len(train_x)


In [ ]:
all_temp[0][1]

In [ ]:
temp_embedding

In [120]:
data_set['news']

0                  증상없이 찾아오는 대장암 예방 어떻게 건강
1         특파원리포트 이대로 집으로 돌아가라고요 지금 우한에선 이런
2               2L 한병에 300원 너무 싼 생수 마셔도 될까
3                   태풍 링링 발생주후반 한반도 영향 가능성
4                  윤석열 칼뺐다조국 청문회날 조국 아내 기소
                       ...                
54895                토스 8억칫솔 무료체험 초성퀴즈 정답은
54896     불화수소 수출 규제 두 달만에 풀었지만 업계 안심하긴 이르
54897       한국 산업 급성장 무서웠나수출로 먹고사는 일본의 자충수
54898        방탄소년단 팰리세이드 타고  빌보드 뮤직 어워드 입성
54899    IFA 2019 LG 야심작 V50S 씽큐 듀얼스크린이 조커
Name: news, Length: 54900, dtype: object

In [173]:
embedding.wv.vocab

{'증상/Noun': <gensim.models.keyedvectors.Vocab at 0x176882ad608>,
 '없이/Adverb': <gensim.models.keyedvectors.Vocab at 0x176882ad908>,
 '찾아오다/Verb': <gensim.models.keyedvectors.Vocab at 0x176882ad4c8>,
 '대장암/Noun': <gensim.models.keyedvectors.Vocab at 0x176882ad3c8>,
 '예방/Noun': <gensim.models.keyedvectors.Vocab at 0x176882ad348>,
 '어떻다/Adjective': <gensim.models.keyedvectors.Vocab at 0x176882ade88>,
 '건강/Noun': <gensim.models.keyedvectors.Vocab at 0x176882adc88>,
 '특파원/Noun': <gensim.models.keyedvectors.Vocab at 0x176882add88>,
 '리포트/Noun': <gensim.models.keyedvectors.Vocab at 0x176882ad988>,
 '이대로/Adverb': <gensim.models.keyedvectors.Vocab at 0x176882adb08>,
 '집/Noun': <gensim.models.keyedvectors.Vocab at 0x176882ad2c8>,
 '으로/Josa': <gensim.models.keyedvectors.Vocab at 0x176882adf08>,
 '돌아가다/Verb': <gensim.models.keyedvectors.Vocab at 0x176882ada08>,
 '지금/Noun': <gensim.models.keyedvectors.Vocab at 0x176882adec8>,
 '우한/Noun': <gensim.models.keyedvectors.Vocab at 0x176882ad848>,
 '에선/Jos

In [121]:
label_set

0        1
1        2
2        5
3        1
4        4
        ..
54895    5
54896    5
54897    5
54898    1
54899    5
Name: label, Length: 54900, dtype: int64

In [ ]:
#data_label = {1:'culture',2:'global',3:'politic',4:'society',5:'economy'}

In [71]:
kkm = Kkma()
kkm.pos(data_set['news'])

TypeError: No matching overloads found for kr.lucypark.kkma.KkmaInterface.morphAnalyzer(Series), options are:
	public java.util.List kr.lucypark.kkma.KkmaInterface.morphAnalyzer(java.lang.String) throws java.lang.Exception

	at JPMethod::findOverload(native\common\jp_method.cpp:242)
	at JPMethod::findOverload(native\common\jp_method.cpp:245)
	at JPMethod::invoke(native\common\jp_method.cpp:253)
	at PyJPMethod::__call__(native\python\pyjp_method.cpp:142)


In [63]:
len(data_set)

54900

In [82]:
for i in range(len(data_set)):
    twitter.pos(data_set['news'][i])

In [80]:
konl = Twitter()
test_string =[

'test 데이터를 얼마나 잘 분류하는지 확인합니다.',
'학습이 epoch 단위로 진행될 때 예측력이 얼마나 상승하는지 ',
'loss에 대한 그래프를 그려서 확인합니다.',
'밑에 코드들은 그래프를 이쁘게 그려주기 위한 코드 입니다.'
]
for row in test_string:
    r = konl.pos(row, norm=True, stem=True)
    print( '=' * 20)
    for txt, post in r:
        print( txt, post)
    print( '=' * 20)



test Alpha
데이터 Noun
를 Josa
얼마나 Noun
자다 Verb
분류 Noun
하다 Verb
확인 Noun
하다 Verb
.;[] Punctuation
학습 Noun
이 Josa
epoch Alpha
단위 Noun
로 Josa
진행 Noun
되다 Verb
때 Noun
예측 Noun
력 Suffix
이 Josa
얼마나 Noun
상승 Noun
하다 Verb
loss Alpha
에 Josa
대한 Noun
그래프 Noun
를 Josa
그리다 Verb
확인 Noun
하다 Verb
. Punctuation
밑 Noun
에 Josa
코드 Noun
들 Suffix
은 Josa
그래프 Noun
를 Josa
이쁘다 Adjective
그리다 Verb
위 Noun
한 Josa
코드 Noun
이다 Adjective
. Punctuation


In [75]:
# encoding: utf-8
import csv
import logging

from konlpy import jvm
from konlpy.tag import Twitter

logger = logging.getLogger(__name__)


def main():
    """
        konlpy 사용시 주의 사항
        자바 설치 및 세팅 필요
        JAVA_HOME 세팅이 필요합니다.
        export JAVA_HOME=$(/usr/libexec/java_home)
    """
    konl = Twitter()
    test_string = [
        u'konlpy 사용시 주의 사항',
        u'자바 설치 및 세팅 필요',
        u'JAVA_HOME 세팅이 필요합니다.',
        u'export JAVA_HOME=$(/usr/libexec/java_home)',
    ]
    for row in test_string:
        r = konl.pos(row, norm=True, stem=True)
        print( '=' * 20)
        for txt, post in r:
            print( txt, post)
        print( '=' * 20)


if __name__ == '__main__':
    jvm.init_jvm()
    main()

konlpy Alpha
사용 Noun
시 Noun
주의 Noun
사항 Noun
자바 Noun
설치 Noun
및 Noun
세팅 Noun
필요 Noun
JAVA Alpha
_ Punctuation
HOME Alpha
세팅 Noun
이 Josa
필요하다 Adjective
. Punctuation
export Alpha
JAVA Alpha
_ Punctuation
HOME Alpha
=$(/ Punctuation
usr Alpha
/ Punctuation
libexec Alpha
/ Punctuation
java Alpha
_ Punctuation
home Alpha
) Punctuation


In [ ]:


model = Word2Vec(data_set['news'], min_count=3, iter=20, size=300,sg=1,
                window = 5)
model.init_sims(replace=True)